In [40]:
from pynq import Overlay
from pynq import MMIO
from time import sleep
from pynq.lib import AxiGPIO
overlay = Overlay('/home/xilinx/pynq/overlays/design/design_1.bit')

In [41]:
#AD5628 COMMADS
REFERENCE_VOLTAGE     = 0b10001000000000000000000000000001
CLEAR_ALL_CHANNELS    = 0b10000011111100000000000000000000
CHANNEL_A_MAX         = 0b10000011000011111111111100000000
CHANNEL_A_MIN         = 0b10000011000000000000000000000000
CHANNEL_A_HALF        = 0b10000011000010000000000000000000
WRITE_COMMAND         = 0b10000000000000000000000000000000
UPDATE_ALL_CHANNELS   = 0b10000110000000000000000011111111


#AXI SPI REGISTERS
XSP_DGIER_OFFSET = 0x1C
XSP_IISR_OFFSET = 0x20
XSP_IIER_OFFSET = 0x28
XSP_SRR_OFFSET = 0x40
XSP_CR_OFFSET = 0x60
XSP_SR_OFFSET = 0x64
XSP_DTR_OFFSET = 0x68
XSP_DRR_OFFSET = 0x6C
XSP_SSR_OFFSET = 0x70
XSP_TFO_OFFSET = 0x74
XSP_RFO_OFFSET = 0x78

#AXI SPI MASKS
XSP_SRR_RESET_MASK = 0x0A
XSP_SR_TX_EMPTY_MASK = 0x04
XSP_SR_TX_FULL_MASK = 0x08
XSP_CR_TRANS_INHIBIT_MASK = 0x100
XSP_CR_LOOPBACK_MASK = 0x01
XSP_CR_ENABLE_MASK = 0x02
XSP_CR_MASTER_MODE_MASK = 0x04
XSP_CR_CLK_POLARITY_MASK = 0x08
XSP_CR_CLK_PHASE_MASK = 0x10
XSP_CR_TXFIFO_RESET_MASK = 0x20
XSP_CR_RXFIFO_RESET_MASK = 0x40
XSP_CR_MANUAL_SS_MASK = 0x80
SLAVE_NO_SELECTION = 0xFFFFFFFF

#SYNC MASK
SYNC_MASK = 0xFFFFFFFF

In [42]:
#AD5628 Configuration
def configure_ad5628(AxiQspi, clk_phase, clk_pol):
    # Reset the SPI device
    AxiQspi.write(XSP_SRR_OFFSET, XSP_SRR_RESET_MASK)
    # Enable the transmit empty interrupt, which we use to determine progress on the transmission. 
    AxiQspi.write(XSP_IIER_OFFSET, XSP_SR_TX_EMPTY_MASK)
    # Disable the global IPIF interrupt
    AxiQspi.write(XSP_DGIER_OFFSET, 0)
    # Deselect the slave on the SPI bus
    AxiQspi.write(XSP_SSR_OFFSET, SLAVE_NO_SELECTION)
    # Disable the transmitter, enable Manual Slave Select Assertion, put SPI controller into master mode, and enable it
    ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg = ControlReg | XSP_CR_MASTER_MODE_MASK | XSP_CR_MANUAL_SS_MASK | XSP_CR_ENABLE_MASK | XSP_CR_TXFIFO_RESET_MASK | XSP_CR_RXFIFO_RESET_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg)
    ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg = ControlReg & ~(XSP_CR_CLK_PHASE_MASK | XSP_CR_CLK_POLARITY_MASK) 
    if clk_phase == 1:
        ControlReg = ControlReg | XSP_CR_CLK_PHASE_MASK
    if clk_pol == 1:
        ControlReg = ControlReg | XSP_CR_CLK_POLARITY_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg)

    return "AD5628 Configured"

In [43]:
#Function for AD5628 Data Transfers
def transfer_ad5628(packet, AxiQspi):
    #print("TransferData")
    for data in packet:
        AxiQspi.write(XSP_DTR_OFFSET, data)
        AxiQspi.write(XSP_SSR_OFFSET, 0xFFFFFFFE)
        ControlReg = AxiQspi.read(XSP_CR_OFFSET)
        ControlReg = ControlReg & ~XSP_CR_TRANS_INHIBIT_MASK
        AxiQspi.write(XSP_CR_OFFSET, ControlReg)

        StatusReg = AxiQspi.read(XSP_SR_OFFSET)
        while (StatusReg & XSP_SR_TX_EMPTY_MASK) == 0:
            StatusReg = AxiQspi.read(XSP_SR_OFFSET)

        #print('XSP_RFO_OFFSET  : 0x{0:08x}'.format(AxiQspi.read(XSP_RFO_OFFSET)))
        ControlReg = AxiQspi.read(XSP_CR_OFFSET)
        ControlReg = ControlReg | XSP_CR_TRANS_INHIBIT_MASK
        AxiQspi.write(XSP_CR_OFFSET, ControlReg)

    AxiQspi.write(XSP_SSR_OFFSET, SLAVE_NO_SELECTION)

    #print("ReadResponse")
    resp = list()
    RxFifoStatus = AxiQspi.read(XSP_SR_OFFSET) & 0x01
    while RxFifoStatus == 0:
        temp = AxiQspi.read(XSP_RFO_OFFSET)
        #print('XSP_RFO_OFFSET  : 0x{0:08x}'.format(temp))
        temp = AxiQspi.read(XSP_DRR_OFFSET)
        #print('XSP_DRR_OFFSET  : 0x{0:08x}'.format(temp))    
        resp.append(temp)
        RxFifoStatus = AxiQspi.read(XSP_SR_OFFSET) & 0x01

    return resp

In [44]:
def set_reference_voltage(sync_pin, ad5628_mod):
    sync_pin.write(0x0, SYNC_MASK)
    transfer_ad5628([REFERENCE_VOLTAGE], ad5628_mod)
    sync_pin.write(0x1, SYNC_MASK)
    return "Reference voltage set" 

In [45]:
def set_channel_A_max(sync_pin, ad5628_mod):
    sync_pin.write(0x0, SYNC_MASK)
    transfer_ad5628([CHANNEL_A_MAX], ad5628_mod)
    sync_pin.write(0x1, SYNC_MASK)
    return "Channel A set to max" 

In [46]:
def write_channel(sync_pin, ad5628_mod, channel, value):
    if (channel > 7 or channel < 0 or value > 2.5 or value < 0):
        return "Write failure: channel range [0,7] value range [0,2.5]"
    channel_int = channel << 20
    value_int = round(4096*(value/2.5))
    value_int = value_int << 8
    command = WRITE_COMMAND | value_int | channel_int
    sync_pin.write(0x0, SYNC_MASK)
    transfer_ad5628([command], ad5628_mod)
    sync_pin.write(0x1, SYNC_MASK)
    return f"Channel {channel} written with value {value}" 

In [47]:
def update_all_channels(sync_pin, ad5628_mod):
    sync_pin.write(0x0, SYNC_MASK)
    transfer_ad5628([UPDATE_ALL_CHANNELS], ad5628_mod)
    sync_pin.write(0x1, SYNC_MASK)
    return "All Channels Updated"

In [48]:
def clear_all_channels(sync_pin,ad5628_mod):
    sync_pin.write(0x0, SYNC_MASK)
    transfer_ad5628([CLEAR_ALL_CHANNELS], ad5628_mod)
    sync_pin.write(0x1, SYNC_MASK)
    return "All Channels Cleared"